In [1]:
import re
import requests
import pandas as pd
from lxml import etree
from tqdm.notebook import tqdm

In [2]:
class Yahoomovie:
    header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'
    }
    tables = []
    panda_tables = {'電影名稱':[],'上映日期':[],'期待度':[],'滿意度':[],'電影簡介':[]}
    
    def __init__(self,start_page,end_page,to_dataframe=True):
        self.start_page = start_page
        self.end_page = end_page
        self.to_dataframe = to_dataframe
        
    def parse_page(self,url):
        response = requests.get(url,headers=self.header)
        html = etree.HTML(response.text)
        movie_name = [i.strip() for i in html.xpath('//div[@class="release_movie_name"]/a[@class="gabtn"]//text()')]
        release_date = html.xpath('//div[@class="release_info"]//div[@class="release_movie_time"]//text()')
        expect = html.xpath('//div[@class="release_info"]//dt//div[@class="leveltext"]//span//text()')
        satisfy = html.xpath('//div[@class="release_info"]//dd//span[@class="count"]/@data-num')
        story = [self.parse_introduce(i) for i in html.xpath('//div[@class="release_info"]//div[@class="en"]//a[@class="gabtn"]/@href')]
        table = {'電影名稱':movie_name,'上映日期':release_date,'期待度':expect,'滿意度':satisfy,'電影簡介':story}
        return table
    
    def parse_introduce(self,link):
        response2 = requests.get(link,headers=self.header)
        html2 = etree.HTML(response2.text)
        words = ''.join(html2.xpath('//span[@id="story"]//text()')).strip()
        return words
        
    def parse_pages(self):
        self.tables = []
        for i in tqdm(range(self.start_page,self.end_page+1)):
            url = 'https://movies.yahoo.com.tw/movie_intheaters.html?page={}'.format(i)
            table = self.parse_page(url)
            self.tables.append(table)
    
    def to_pandas(self):
        self.panda_tables = {'電影名稱':[],'上映日期':[],'期待度':[],'滿意度':[],'電影簡介':[]}
        for table in self.tables:
            self.panda_tables['電影名稱'].extend(table['電影名稱'])
            self.panda_tables['上映日期'].extend(table['上映日期'])
            self.panda_tables['期待度'].extend(table['期待度'])
            self.panda_tables['滿意度'].extend(table['滿意度'])
            self.panda_tables['電影簡介'].extend(table['電影簡介'])
        self.panda_tables = pd.DataFrame(self.panda_tables)
        self.panda_tables['上映日期'] = self.panda_tables['上映日期'].str.split(' ').str[-1]
        self.panda_tables['期待度'] = self.panda_tables['期待度'].str.replace('%','').astype('int')
        self.panda_tables['滿意度'] = self.panda_tables['滿意度'].astype('float')
        self.panda_tables['電影簡介'] = self.panda_tables['電影簡介'].apply(lambda x : re.sub(r'\s','',x))
        
    def run(self):
        self.parse_pages()
        if self.to_dataframe:
            self.to_pandas()

In [3]:
data=Yahoomovie(1,10)
data.run()

In [4]:
data.panda_tables

,電影名稱,上映日期,期待度,滿意度,電影簡介
0,大說謊家,2019-12-13,91,3.7,故事主角是一個不曾相信過任何人的職業騙徒：羅伊寇特尼(伊恩麥克連飾)，他一生中無所不騙，心狠...
1,熱氣球飛行家,2019-12-13,85,4.2,★金獎影帝艾迪瑞德曼暫時脫下魔法袍回歸真實演技★《愛的萬物論》霍金夫婦檔再次聯手演繹真實人物...
2,黑色聖誕節,2019-12-13,55,1.9,改編自經典恐怖片的《黑色聖誕節》是一部符合佳節氣氛的全新恐怖片，故事描述一名校園連環殺手，即...
3,為你存在的每一天,2019-12-13,80,4.1,★今年冬天最動容的溫馨電影！★榮獲洛杉磯電影獎《最佳劇情片》《最佳新導演》《最佳演員組合》《...
4,8級警戒,2019-12-13,95,4.3,★《刺激驚爆點》製片打造繼《成人世界》後又一寫實科幻作品★《環太平洋》《大亨小傳》幕後團隊傾...
5,每個月來訪一次的月經醬,2019-12-13,33,3.0,★手塚治虫文化賞短篇獎得主點擊超越2000萬★廣大女孩共感度No.1壓倒性支持關於每個月的那...
6,巴黎小情聖,2019-12-13,78,4.0,★見證《大鼻子情聖》傳奇誕生好評如潮轟動各大影展★鬼才導演艾歷克西斯·米奇利克全新展現19世...
7,假面騎士劇場版ZI-O Over Quartzer,2019-12-13,67,4.0,★假面騎士平成系列的第二十部作品，同時也是平成系列的最後一部作品。★史上首次結合6款假面騎士...
8,HUMAN LOST人間失格,2019-12-13,74,2.9,通過醫療改革，在未來的東京人類基本可達到無病長壽的境界！於人體內植入芯片，利用網路對此進行管...
9,黑計畫,2019-12-13,59,3.2,★震驚韓國史上最大金融弊案精彩緊湊直逼《大賣空》★小人物對抗大財團揭開暗黑內幕鈔級騙局一觸即...
